In [ ]:
%matplotlib inline
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import ottawa_libraries as ol
import pulling_library_info

In [ ]:
def get_map_data(city):
    if city == 'Ottawa': city = "Ottawa - Gatineau (Ontario part / partie de l'Ontario)"
    elif city == 'Montreal': city = 'Montréal'
    census_file = 'data/census_tracts.shp'
    canada_census = gpd.read_file(census_file)
    city_census = canada_census[(canada_census.CMANAME==city)]
    city_census = city_census.to_crs('epsg:4326')
    return city_census

In [ ]:
def format_library_data(library_data):
    libraries = pd.DataFrame(library_data)
    libraries = libraries.T.reset_index()
    libraries = libraries.rename(columns={'index':'Name'})
    libraries['Latitude'] = pd.to_numeric(libraries['Latitude'])
    libraries['Longitude'] = pd.to_numeric(libraries['Longitude'])
    libraries['geometry'] = libraries.apply(lambda x:
                                                shapely.geometry.point.Point(x['Longitude'], x['Latitude']),
                                                axis=1)
    libraries_gpd = gpd.GeoDataFrame(libraries)
    return libraries_gpd

In [ ]:
def add_in_density(map_data, city):
    pop_data = pd.read_csv('data/98-401-X2016043_eng_CSV/98-401-X2016043_English_CSV_data.csv')
    city_pop_data = pop_data[pop_data['GEO_CODE (POR)'].isin(map_data.CTUID)]
    city_pop_data = city_pop_data.loc[city_pop_data["DIM: Profile of Census Tracts (2247)"]=='Population, 2016']
    city_pop_data['population'] = city_pop_data['Dim: Sex (3): Member ID: [1]: Total - Sex']
    # resolves issue with 2 census tracts in Montreal (doesn't impact any of the other top cities)
    # TODO: see if '..' codes to something
    city_pop_data['population'] = city_pop_data['population'].replace('..', 0)
    
    city_pop_data['population'] = pd.to_numeric(city_pop_data['population'])

    map_data['CTUID'] = pd.to_numeric(map_data['CTUID'])
    map_data = map_data.merge(right=city_pop_data, how='left',
                                  left_on='CTUID', right_on='GEO_CODE (POR)')
    map_data['density'] = map_data['population']/map_data.area
    return map_data

In [ ]:
def pipeline(city):
    library_data = pulling_library_info.main(city)
    libraries_gpd = format_library_data(library_data)
    map_data = get_map_data(city)
    map_data = add_in_density(map_data, city)
    map_data_display = map_data[['geometry', 'population', 'density']]
    m = map_data_display.explore(column='density')
    libraries_gpd.explore(m=m, color='red')
    print(m.crs)
    display(m)
    return m

In [ ]:
m_ottawa = pipeline('Ottawa')

In [ ]:
type(m_ottawa)

In [ ]:
m_ottawa.crs

In [ ]:
m_calgary = pipeline('Calgary')

In [ ]:
m_ottawa

In [ ]:
m_edmonton = pipeline('Edmonton')

In [ ]:
m_toronto = pipeline('Toronto')

In [ ]:
m_montreal = pipeline('Montreal')